In [1]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
import pandas as pd
import numpy as np
from itertools import chain, combinations
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
from memory_profiler import memory_usage

import time

# Record the start time
start_time = time.time()

In [2]:
# Function to generate all subsets of a set
# def powerset(s):
#     return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [3]:
# Read the contents of the file
file_path = 'kddtrain+.txt'
with open(file_path, 'r') as file:
    lines = file.readlines()

In [4]:
# Process rows and split each row into a list of items
transactions = [line.strip().split(',') for line in lines]
transactions

[['0',
  'tcp',
  'ftp_data',
  'SF',
  '491',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '2',
  '2',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '150',
  '25',
  '0.17',
  '0.03',
  '0.17',
  '0.00',
  '0.00',
  '0.00',
  '0.05',
  '0.00',
  'normal',
  '20'],
 ['0',
  'udp',
  'other',
  'SF',
  '146',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '13',
  '1',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.08',
  '0.15',
  '0.00',
  '255',
  '1',
  '0.00',
  '0.60',
  '0.88',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  'normal',
  '15'],
 ['0',
  'tcp',
  'private',
  'S0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '123',
  '6',
  '1.00',
  '1.00',
  '0.00',
  '0.00',
  '0.05',
  '0.07',
  '0.00',
  '255',
  '26',
 

In [5]:
# Convert the list of transactions into a DataFrame
df = pd.DataFrame(transactions)
df

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune,20
125969,8,udp,private,SF,105,145,0,0,0,0,...,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal,21
125970,0,tcp,smtp,SF,2231,384,0,0,0,0,...,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal,18
125971,0,tcp,klogin,S0,0,0,0,0,0,0,...,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,20


In [6]:
# Assuming df is your original DataFrame
# Select the desired columns by their integer position
selected_columns = [1, 2, 3, 41]

# Create a new DataFrame with only the selected columns
df = df.iloc[:, selected_columns].copy()
df = df.sort_index(axis=1)
df
sampled_df = df

In [7]:
# Set the sampling rate (e.g., 0.5 for 50%)
#sampling_rate = 0.005

In [8]:
# Randomly sample a subset of the dataset
#sampled_df = df.sample(frac=sampling_rate, random_state=42)
#sampled_df

In [9]:
# Initialize an empty list to store the items
updated_transactions = []

# Iterate over the rows of the DataFrame
for index, row in sampled_df.iterrows():
    # Split each row into items and append them to the items_list
    items = row.tolist()  # Convert the row to a list
    updated_transactions.append(items)

updated_transactions

[['tcp', 'ftp_data', 'SF', 'normal'],
 ['udp', 'other', 'SF', 'normal'],
 ['tcp', 'private', 'S0', 'neptune'],
 ['tcp', 'http', 'SF', 'normal'],
 ['tcp', 'http', 'SF', 'normal'],
 ['tcp', 'private', 'REJ', 'neptune'],
 ['tcp', 'private', 'S0', 'neptune'],
 ['tcp', 'private', 'S0', 'neptune'],
 ['tcp', 'remote_job', 'S0', 'neptune'],
 ['tcp', 'private', 'S0', 'neptune'],
 ['tcp', 'private', 'REJ', 'neptune'],
 ['tcp', 'private', 'S0', 'neptune'],
 ['tcp', 'http', 'SF', 'normal'],
 ['tcp', 'ftp_data', 'SF', 'warezclient'],
 ['tcp', 'name', 'S0', 'neptune'],
 ['tcp', 'netbios_ns', 'S0', 'neptune'],
 ['tcp', 'http', 'SF', 'normal'],
 ['icmp', 'eco_i', 'SF', 'ipsweep'],
 ['tcp', 'http', 'SF', 'normal'],
 ['tcp', 'http', 'SF', 'normal'],
 ['tcp', 'mtp', 'S0', 'neptune'],
 ['tcp', 'private', 'S0', 'neptune'],
 ['tcp', 'http', 'SF', 'normal'],
 ['udp', 'other', 'SF', 'normal'],
 ['tcp', 'mtp', 'S0', 'neptune'],
 ['tcp', 'telnet', 'SF', 'normal'],
 ['tcp', 'private', 'S0', 'neptune'],
 ['tcp', 

In [10]:
# Assuming you have a list of transactions named transactions
num_transactions = len(updated_transactions)
print("Number of transactions:", num_transactions)

Number of transactions: 125973


In [11]:
# Convert the list of transactions into a one-hot encoded format
te = TransactionEncoder()
te_ary = te.fit(updated_transactions).transform(updated_transactions)
sampled_encoded_df = pd.DataFrame(te_ary, columns=te.columns_)
sampled_encoded_df

,IRC,OTH,REJ,RSTO,RSTOS0,RSTR,S0,S1,S2,S3,...,time,udp,urh_i,urp_i,uucp,uucp_path,vmnet,warezclient,warezmaster,whois
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
125969,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
125970,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
125971,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [12]:
min_support = 1e-10

# Total number of transactions
total_transactions = len(updated_transactions)

In [13]:
# Run FP-growth algorithm to mine frequent itemsets on the sampled dataset
frequent_itemsets = fpgrowth(sampled_encoded_df, min_support, use_colnames=True)
# Round the support values to avoid precision issues
#frequent_itemsets['support_rounded'] = frequent_itemsets['support']

# Multiply the support counts by the total number of transactions to get absolute support counts
frequent_itemsets['support'] = frequent_itemsets['support'] * total_transactions

# Print frequent itemsets
print("Frequent Itemsets:")
print(frequent_itemsets)

#frequent_itemsets['support'] *= 10
# frequent_itemsets

Frequent Itemsets:
       support                    itemsets
0     102689.0                       (tcp)
1      74945.0                        (SF)
2      67343.0                    (normal)
3       6860.0                  (ftp_data)
4      14993.0                       (udp)
...        ...                         ...
2816       1.0        (S0, satan, harvest)
2817       1.0   (S0, tcp, satan, harvest)
2818       1.0         (tcp, harvest, REJ)
2819       1.0       (satan, harvest, REJ)
2820       1.0  (tcp, satan, harvest, REJ)

[2821 rows x 2 columns]


In [14]:
# Declare updated_min_sup
updated_min_sup = 350

# Keep frequent itemsets with support >= updated_min_sup
ultimate_freq_itemsets = frequent_itemsets[frequent_itemsets['support'] >= updated_min_sup]

# Keep frequent itemsets with support < updated_min_sup
ultimate_infreq_itemsets = frequent_itemsets[frequent_itemsets['support'] < updated_min_sup]

In [15]:
# Print ultimate_freq_itemsets
print("Frequent Itemsets with Support >= updated_min_sup:")
print("ultimate frequent itemsets: ")
print(ultimate_freq_itemsets)
len(ultimate_freq_itemsets)

# Print ultimate_infreq_itemsets
print("\nInfrequent Itemsets with Support < updated_min_sup:")
print("ultimate infrequent itemsets: ")
print(ultimate_infreq_itemsets)
print(ultimate_infreq_itemsets)

Frequent Itemsets with Support >= updated_min_sup:
ultimate frequent itemsets: 
       support                  itemsets
0     102689.0                     (tcp)
1      74945.0                      (SF)
2      67343.0                  (normal)
3       6860.0                (ftp_data)
4      14993.0                     (udp)
...        ...                       ...
2192     622.0      (neptune, tcp, nnsp)
2193     498.0       (neptune, S0, nnsp)
2194     498.0           (tcp, S0, nnsp)
2195     498.0  (neptune, tcp, S0, nnsp)
2445     362.0        (netbios_ssn, tcp)

[506 rows x 2 columns]

Infrequent Itemsets with Support < updated_min_sup:
ultimate infrequent itemsets: 
      support                    itemsets
11       78.0                (remote_job)
14      347.0                (netbios_ns)
43      271.0                        (SH)
53      201.0                       (pod)
57      299.0                    (kshell)
...       ...                         ...
2816      1.0        (S0, 

In [16]:
# Extract itemsets with only one item from ultimate_freq_itemsets
one_length_freq_items = ultimate_freq_itemsets[ultimate_freq_itemsets['itemsets'].apply(lambda x: len(x)) == 1]

# Extract itemsets with only one item from ultimate_infreq_itemsets
one_length_infreq_items = ultimate_infreq_itemsets[ultimate_infreq_itemsets['itemsets'].apply(lambda x: len(x)) == 1]

# Concatenate the one-length itemsets into a single DataFrame
one_length_items = pd.concat([one_length_freq_items, one_length_infreq_items])

# Print the updated support count for one-length items
print("One-Length Items with Support Count:")
print(one_length_items)

One-Length Items with Support Count:
      support     itemsets
0    102689.0        (tcp)
1     74945.0         (SF)
2     67343.0     (normal)
3      6860.0   (ftp_data)
4     14993.0        (udp)
..        ...          ...
102       2.0        (aol)
103       1.0  (http_2784)
104       3.0     (tftp_u)
105       3.0       (perl)
106       2.0    (harvest)

[107 rows x 2 columns]


In [17]:
###########################################################################

In [18]:
# Filter out infrequent itemsets with support <= 4
#infrequent_itemsets = frequent_itemsets[frequent_itemsets['support_rounded'] <= min_support]
#infrequent_itemsets

In [19]:
# Filter infrequent itemsets with more than one item
infrequent_itemsets_more_than_one = ultimate_infreq_itemsets[ultimate_infreq_itemsets['itemsets'].apply(lambda x: len(x)) > 1]
infrequent_itemsets_more_than_one

,support,itemsets
116,114.0,"(ftp_data, REJ)"
125,114.0,"(ftp_data, tcp, REJ)"
126,79.0,"(neptune, ftp_data, REJ)"
127,79.0,"(neptune, ftp_data, tcp, REJ)"
140,109.0,"(other, S0)"
...,...,...
2816,1.0,"(S0, satan, harvest)"
2817,1.0,"(S0, tcp, satan, harvest)"
2818,1.0,"(tcp, harvest, REJ)"
2819,1.0,"(satan, harvest, REJ)"


In [20]:
# Define a function to calculate support count for an itemset
# def support_count(itemset, transactions):
#     count = 0
#     for transaction in transactions:
#         if itemset.issubset(transaction):
#             count += 1
#     return count

In [21]:
# Threshold for anti-monotone value
h_conf = 0.002

# List to store itemsets with anti-monotone value >= h_conf
itemsets_with_high_antimonotone = []

# Iterate over infrequent_itemsets_more_than_one
for _, row in infrequent_itemsets_more_than_one.iterrows():
    itemset = row['itemsets']
    
    # Calculate support count for the itemset
    sup_count = row['support']
    
    # Find maximum support count among individual items in the itemset
    max_sup_count = max(one_length_items.loc[one_length_items['itemsets'].apply(lambda x: x.issubset(itemset)), 'support'])
    
    # Calculate anti-monotone value
    anti_monotone = (sup_count / max_sup_count)

    # Round the anti-monotone value to two decimal points
    anti_monotone = round(anti_monotone, 2)

    # Check if anti-monotone value is greater than or equal to the threshold
    if anti_monotone >= h_conf:
        # Append the itemset to the list
        itemsets_with_high_antimonotone.append((itemset, sup_count, anti_monotone))

# Print the itemsets with anti-monotone value >= h_conf
for itemset_data in itemsets_with_high_antimonotone:
    print(f"Itemset: {itemset_data[0]}, Support Count: {itemset_data[1]}, Anti-Monotone Value: {itemset_data[2]}")

Itemset: frozenset({'ftp_data', 'REJ'}), Support Count: 114.0, Anti-Monotone Value: 0.01
Itemset: frozenset({'RSTOS0', 'remote_job'}), Support Count: 1.0, Anti-Monotone Value: 0.01
Itemset: frozenset({'warezclient', 'ftp'}), Support Count: 306.0, Anti-Monotone Value: 0.17
Itemset: frozenset({'name', 'S0'}), Support Count: 331.0, Anti-Monotone Value: 0.01
Itemset: frozenset({'name', 'REJ'}), Support Count: 110.0, Anti-Monotone Value: 0.01
Itemset: frozenset({'neptune', 'name', 'S0'}), Support Count: 330.0, Anti-Monotone Value: 0.01
Itemset: frozenset({'neptune', 'netbios_ns'}), Support Count: 336.0, Anti-Monotone Value: 0.01
Itemset: frozenset({'S0', 'netbios_ns'}), Support Count: 263.0, Anti-Monotone Value: 0.01
Itemset: frozenset({'netbios_ns', 'REJ'}), Support Count: 77.0, Anti-Monotone Value: 0.01
Itemset: frozenset({'neptune', 'S0', 'netbios_ns'}), Support Count: 262.0, Anti-Monotone Value: 0.01
Itemset: frozenset({'private', 'ipsweep'}), Support Count: 331.0, Anti-Monotone Value: 

In [22]:
# Function to calculate cosine similarity for an itemset
def cosine_similarity(itemset, infrequent_itemsets_more_than_one, one_length_items):
    # If the itemset contains only one item, return 1 as cosine similarity
    if len(itemset) == 1:
        return 1.0
    
    # Initialize variables to store support counts
    support_ab = 0
    support_a = 0
    support_b = 0
    
    # Find support counts from infrequent_itemsets_more_than_one
    for _, row in infrequent_itemsets_more_than_one.iterrows():
        if row['itemsets'] == itemset:
            support_ab = row['support']
            break
    
    # Find support counts from one_length_items for each item in the pair
    for pair in combinations(itemset, 2):
        item_a = pair[0]
        item_b = pair[1]
        for _, row in one_length_items.iterrows():
            if {item_a}.issubset(row['itemsets']):
                support_a = row['support']
            if {item_b}.issubset(row['itemsets']):
                support_b = row['support']

    #print(item_a)
    #print(support_a)
    #print(item_b)
    #print(support_b)
    
    # Calculate cosine similarity
    cosine_ab = support_ab / np.sqrt(support_a * support_b)
    
    return cosine_ab

# Function to calculate the average cosine similarity for an itemset
def average_cosine_similarity(itemset, infrequent_itemsets_more_than_one, one_length_items):
    # If the itemset contains only one item, return 1 as cosine similarity
    if len(itemset) == 1:
        return 1.0
    
    # Initialize a list to store cosine similarities for each pair of items
    similarities = []
    
    # Calculate cosine similarity for each pair of items in the itemset
    for pair in combinations(itemset, 2):
        cosine_sim = cosine_similarity(set(pair), infrequent_itemsets_more_than_one, one_length_items)
        similarities.append(cosine_sim)
    
    # Calculate the average cosine similarity
    average_cosine = np.mean(similarities)
    
    return average_cosine

# Example usage
#itemset = {'item1', 'item2', 'item3'}
#avg_cosine_sim = average_cosine_similarity(itemset, infrequent_itemsets_more_than_one, one_length_items)
#print("Average Cosine Similarity:", avg_cosine_sim)

In [23]:
# Function to calculate Jaccard similarity for an itemset
def jaccard_similarity(itemset, infrequent_itemsets_more_than_one, one_length_items):
    # If the itemset contains only one item, return 1 as Jaccard similarity
    if len(itemset) == 1:
        return 1.0
    
    # Initialize variables to store support counts
    support_ab = 0
    support_a = 0
    support_b = 0
    
    # Find support counts from infrequent_itemsets_more_than_one
    for _, row in infrequent_itemsets_more_than_one.iterrows():
        if row['itemsets'] == itemset:
            support_ab = row['support']
            break
    
    # Find support counts from one_length_items for each item in the pair
    for pair in combinations(itemset, 2):
        item_a = pair[0]
        item_b = pair[1]
        for _, row in one_length_items.iterrows():
            if {item_a}.issubset(row['itemsets']):
                support_a = row['support']
            if {item_b}.issubset(row['itemsets']):
                support_b = row['support']
    
    # Calculate Jaccard similarity
    jaccard_ab = support_ab / (support_a + support_b - support_ab)
    
    return jaccard_ab

# Function to calculate the average Jaccard similarity for an itemset
def average_jaccard_similarity(itemset, infrequent_itemsets_more_than_one, one_length_items):
    # If the itemset contains only one item, return 1 as Jaccard similarity
    if len(itemset) == 1:
        return 1.0
    
    # Initialize a list to store Jaccard similarities for each pair of items
    similarities = []
    
    # Calculate Jaccard similarity for each pair of items in the itemset
    for pair in combinations(itemset, 2):
        jaccard_sim = jaccard_similarity(set(pair), infrequent_itemsets_more_than_one, one_length_items)
        similarities.append(jaccard_sim)
    
    # Calculate the average Jaccard similarity
    average_jaccard = np.mean(similarities)
    
    return average_jaccard

# Example usage
#itemset = {'item1', 'item2', 'item3'}
#avg_jaccard_sim = average_jaccard_similarity(itemset, infrequent_itemsets_more_than_one, one_length_items)
#print("Average Jaccard Similarity:", avg_jaccard_sim)

In [24]:
# Print itemsets that satisfy both cosine and Jaccard similarity conditions
corr_anomalous = 0

for itemset_data in itemsets_with_high_antimonotone:
    itemset = itemset_data[0]
    cosine_value = average_cosine_similarity(itemset, infrequent_itemsets_more_than_one, one_length_items).round(2)
    jaccard_value = average_jaccard_similarity(itemset, infrequent_itemsets_more_than_one, one_length_items).round(2)
    anti_monotone_value = itemset_data[2]
    if cosine_value >= anti_monotone_value and jaccard_value >= anti_monotone_value / 2:
        corr_anomalous += 1
        print(f"Itemset: {itemset}, Cosine Similarity: {cosine_value}, Jaccard Similarity: {jaccard_value}")

Itemset: frozenset({'ftp_data', 'REJ'}), Cosine Similarity: 0.01, Jaccard Similarity: 0.01
Itemset: frozenset({'RSTOS0', 'remote_job'}), Cosine Similarity: 0.01, Jaccard Similarity: 0.01
Itemset: frozenset({'warezclient', 'ftp'}), Cosine Similarity: 0.24, Jaccard Similarity: 0.13
Itemset: frozenset({'name', 'S0'}), Cosine Similarity: 0.08, Jaccard Similarity: 0.01
Itemset: frozenset({'name', 'REJ'}), Cosine Similarity: 0.05, Jaccard Similarity: 0.01
Itemset: frozenset({'neptune', 'netbios_ns'}), Cosine Similarity: 0.09, Jaccard Similarity: 0.01
Itemset: frozenset({'S0', 'netbios_ns'}), Cosine Similarity: 0.08, Jaccard Similarity: 0.01
Itemset: frozenset({'netbios_ns', 'REJ'}), Cosine Similarity: 0.04, Jaccard Similarity: 0.01
Itemset: frozenset({'neptune', 'S0', 'netbios_ns'}), Cosine Similarity: 0.05, Jaccard Similarity: 0.01
Itemset: frozenset({'private', 'ipsweep'}), Cosine Similarity: 0.04, Jaccard Similarity: 0.01
Itemset: frozenset({'S0', 'mtp'}), Cosine Similarity: 0.09, Jaccard

In [25]:
print("Number of correlated anomalous itemsets is:")
corr_anomalous

Number of correlated anomalous itemsets is:


108

In [26]:
# Record the end time
end_time = time.time()

# Calculate the total time taken
total_time = end_time - start_time

# Print the total time
print("Total time taken:", total_time, "seconds")

Total time taken: 20.465870141983032 seconds


In [27]:
# Memory profiler
mem_usage = memory_usage(-1, interval=1)
max_mem_usage = max(mem_usage)
print("Maximum memory usage:", max_mem_usage, "MiB")

Maximum memory usage: 486.75390625 MiB


In [28]:
# Call garbage collector to free up memory
import gc
gc.collect()

8398